In [1]:
!pip install bs4
!pip install html5lib
!pip install requests
!pip install psycopg2-binary
!pip install geopy

You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\leonp\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from IPython.display import clear_output
import json
from geopy.geocoders import Nominatim

In [3]:
# load data from json
with open('options.json') as file:
    options = json.loads(file.read())

detailsTable = options['detailsTable']
eqOptionsTable = options['eqOptionsTable']
eqOptions = options['eqOptions']
mainTable = options['mainTable']
mainInfoTable = options['mainInfoTable']
equipmentTable = options['equipmentTable']
sellerTable = options['sellerTable']
descriptionTable = options['descriptionTable']
priceTable = options['priceTable']
locationTable = options['locationTable']

In [4]:
def getOffersUrlsFromPage(url):
    pageContent = requests.get(url).content

    soup = BeautifulSoup(pageContent, 'html5lib')

    for i in soup.find_all('a', attrs={'class': 'offer-title__link'}):
        if not i.has_attr('data-test'):
            yield i['href']

In [5]:
def getDetailsFromOffer(soup):
    result = {}
    for parameter in soup.find_all('li', attrs={'class': 'offer-params__item'}):
        key = parameter.find('span').text.replace(' ', '_').replace('(', '').replace(')', '')
        try:
            value = parameter.find('div').text.strip().replace("'", '')
        except AttributeError:
            continue

        if key in detailsTable:
            if detailsTable[key] != -1:
                value = value[:-detailsTable[key]].replace(' ', '').replace(',', '.')
        else:
            continue
        
        result[key] = value
    return result

In [6]:
def getIDFromOffer(soup):
    try:
        offerID = soup.find('span', attrs={'id': 'ad_id'}).text
    except AttributeError:
        return None
    return {'offer_id': offerID}

In [7]:
def getSellerFromOffer(soup):
    sellerName = soup.find('h2', attrs={'class': 'seller-heading__seller-name'}).text.strip().replace("'", "''")
    sellerType = soup.find('section', attrs={'class': 'seller-features'}).find('article', attrs={'class': 'seller-features__feature'}).text.strip().replace("'", "''")

    return {'seller_name': sellerName, 'seller_type': sellerType}

In [8]:
def getPriceFromOffer(soup):
    priceCurrency = soup.find('span', attrs={'class': 'offer-price__number'}).text
    price, currency = map(lambda x: x.strip().replace(',', '.'), priceCurrency.replace(' ', '', 2).split(' ', 1))
    forNegotiation = soup.find('span', attrs={'class': 'offer-price__details'}).text

    return {'price': price, 'currency': currency, 'offer_type': forNegotiation}

In [9]:
def getDescFromOffer(soup):
    return {'description': soup.find('div', attrs={'class': 'offer-description__description'}).text.replace("'", "''")}

In [10]:
def getEquipmentFromOffer(soup):
    result = { i: False for i in eqOptionsTable}
    try:
        offerEq = soup.find('div', attrs={'class': 'offer-features'}).text
    except AttributeError:
        return result

    for (i, j) in zip(eqOptions, eqOptionsTable):
        if i in offerEq:
            result[j] = True

    return result

In [11]:
def getLocationFromOffer(soup):
    locationStr = soup.find('article', attrs={'class': 'seller-card__links__link seller-card__links__link--address'}).text

    geolocator = Nominatim(user_agent='request')
    location = geolocator.geocode(locationStr)

    if location is None or location.longitude is None or location.latitude is None:
        return {}
    else:
        return {'longitude': location.longitude, 'latitude': location.latitude}

In [12]:
def dropTable(cursor):
    cursor.execute('DROP TABLE IF EXISTS details')
    cursor.execute('DROP TABLE IF EXISTS equipment')
    cursor.execute('DROP TABLE IF EXISTS seller')
    cursor.execute('DROP TABLE IF EXISTS description')
    cursor.execute('DROP TABLE IF EXISTS price')
    cursor.execute('DROP TABLE IF EXISTS location')
    cursor.execute('DROP TABLE IF EXISTS offers')

In [13]:
def createTables(cursor):
    cursor.execute(mainTable)
    cursor.execute(mainInfoTable)
    cursor.execute(equipmentTable)
    cursor.execute(sellerTable)
    cursor.execute(descriptionTable)
    cursor.execute(priceTable)
    cursor.execute(locationTable)

In [14]:
def insertData(cursor, tableName, data):
    insertDataCommand = f'INSERT INTO {tableName} ('

    for col in data:
        insertDataCommand += f'{col}, '
    insertDataCommand = insertDataCommand[:-2] + ') VALUES ('

    for value in data.values():
        insertDataCommand += f"'{value}', "
    insertDataCommand = insertDataCommand[:-2] + ') ON CONFLICT DO NOTHING'

    cursor.execute(insertDataCommand)

In [15]:
sql = psycopg2.connect(host='192.168.10.163', port='5433', database='otomoto-1', user='leon', password='biznes')
cur = sql.cursor()

dropTable(cur)

createTables(cur)
sql.commit()

In [16]:
firstLaunch = True
columns = []

counter = 0
for i in range(1, 501):
    for offerUrl in getOffersUrlsFromPage(f'https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={i}'):
        print(offerUrl)
        try:
            content = requests.get(offerUrl).content
        except:
            continue
        soup = BeautifulSoup(content, 'html5lib')

        if soup.find('div', attrs={'class': 'om-page404 adNotFound'}) is not None:
            continue

        offerID = getIDFromOffer(soup)
        if offerID is None:
            continue

        equip = getEquipmentFromOffer(soup)
        details = getDetailsFromOffer(soup)
        seller = getSellerFromOffer(soup)
        money = getPriceFromOffer(soup)
        desc = getDescFromOffer(soup)
        loc = getLocationFromOffer(soup)
        
        insertData(cur, 'offers', offerID | {'url': offerUrl})
        insertData(cur, 'details', offerID | details)
        insertData(cur, 'equipment', offerID | equip)
        insertData(cur, 'price', offerID | money)
        insertData(cur, 'seller', offerID | seller)
        insertData(cur, 'description', offerID | desc)
        insertData(cur, 'location', offerID | loc)
        
        clear_output(wait=True)
        print(counter)
        counter += 1

4031
https://www.otomoto.pl/oferta/audi-q7-rezerwacja-ID6E0qZZ.html#65e0e474fc


In [ ]:
sql.commit()
cur.close()
sql.close()